<a href="https://colab.research.google.com/github/bhargavreddy111/Fraud-Detection/blob/master/FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
trainfile = r'/gdrive/My Drive/Fraud Detection/Insurance Fraud - TRAIN-3000(1).csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/Fraud Detection/Insurance Fraud -TEST-12900(1).csv'
testData = pd.read_csv(testfile) #creates a dataframe

#print sizes (shape) of datasets
print(trainData.shape)
print(testData.shape)

trainData.head()
testData.head()

In [0]:
#Copy Train data excluding target
trainData_Copy = trainData.iloc[:, :-1].copy()
testData_Copy = testData.iloc[:, :-1].copy()

trainData_Copy.head()

In [0]:
#List of Categorical Features
categoricalFeatures = ["MONTH", "DAYOFWEEK", 'MAKE', "ACCIDENTAREA", "DAYOFWEEKCLAIMED", "MONTHCLAIMED", 'SEX', 'MARITALSTATUS', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'BASEPOLICY']

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)

X_test.head()

In [0]:
y_train = trainData["FRAUDFOUND"]
#X_train = trainData.drop(["FRAUDFOUND"], axis=1) #extracting training data without the target column
y_test = testData["FRAUDFOUND"]
#X_test = testData.drop(["FRAUDFOUND"], axis=1) #extracting training data without the target column

In [0]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (testing) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print(classification_report(y_test,clf_predict))

accuracy Score (testing) for Decision Tree:0.880864
Confusion Matrix for Decision Tree
[[10934  1486]
 [   53   445]]
              precision    recall  f1-score   support

          No       1.00      0.88      0.93     12420
         Yes       0.23      0.89      0.37       498

    accuracy                           0.88     12918
   macro avg       0.61      0.89      0.65     12918
weighted avg       0.97      0.88      0.91     12918



In [0]:
#Hyperparameter tuning done for decision tree classifier
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(10,40,5),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15,cv=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#GRID SEARCH----------------------------------------
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters,cv=5)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)


#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("accuracy Score (testing) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("accuracy Score (testing) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))
print(classification_report(y_test,clfr_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

accuracy Score (training) after hypertuning randomized search for Decision Tree:0.891631
accuracy Score (testing) after hypertuning randomized search for Decision Tree:0.884115
accuracy Score (testing) after hypertuning grid search for Decision Tree:0.877380
Confusion Matrix after hypertuning for Decision Tree
[[11255  1165]
 [  332   166]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.91      0.94     12420
         Yes       0.12      0.33      0.18       498

    accuracy                           0.88     12918
   macro avg       0.55      0.62      0.56     12918
weighted avg       0.94      0.88      0.91     12918

              precision    recall  f1-score   support

          No       0.97      0.90      0.93     12420
         Yes       0.12      0.35      0.18       498

    accuracy                           0.88     12918
   macro avg       0.55      0.62      0.56     12918
weighted avg       0.94     

In [0]:
#Normal randomforest
rand_parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(1,20,2),'max_features':[2,3,4],'n_estimators':[10,20,30]}
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (testing) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print(classification_report(y_test,rfc_predict))

#RANDOMIZED SEARCH----------------------------------------
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=10,cv=5)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc1= RandomForestClassifier(**grid_parm_rfc)
rfc1.fit(X_train,y_train)
rfc1_predict = rfc1.predict(X_test)
print("accuracy Score (testing_Random Search) after hypertuning for Random Forest:{0:6f}".format(rfc1.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc1_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc1_predict))
rfc1_cv_score = cross_val_score(rfc1, X_train, y_train)
print(rfc1_cv_score)
print('\n')



accuracy Score (testing) for RandomForest:0.958972
Confusion Matrix for Random Forest:
[[12006   414]
 [  116   382]]
              precision    recall  f1-score   support

          No       0.99      0.97      0.98     12420
         Yes       0.48      0.77      0.59       498

    accuracy                           0.96     12918
   macro avg       0.74      0.87      0.78     12918
weighted avg       0.97      0.96      0.96     12918

{'n_estimators': 30, 'min_samples_leaf': 75, 'max_features': 2, 'max_depth': 7}
accuracy Score (testing_Random Search) after hypertuning for Random Forest:0.961449
Confusion Matrix after hypertuning for Random Forest:
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.96      1.00      0.98     12420
         Yes       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weig

In [0]:
#Grid SEARCH----------------------------------------
grid_parameters={'min_samples_leaf' : range(15,90,10),'max_depth': range(1,20,2),'max_features':[2,3,4],'n_estimators':[10,20,30]}
rfc_grid = GridSearchCV(rfc,grid_parameters,cv=5)
rfc_grid.fit(X_train, y_train)
grid_parm_rfc1=rfc_grid.best_params_
print(grid_parm_rfc1)
rfc2= RandomForestClassifier(**grid_parm_rfc1)
rfc2.fit(X_train,y_train)
rfc2_predict = rfc2.predict(X_test)
print("accuracy Score (testing_gridsearch) after hypertuning for Random Forest:{0:6f}".format(rfc2.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc2_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc2_predict))
rfc2_cv_score = cross_val_score(rfc2, X_train, y_train)
print(rfc2_cv_score)
print('\n')

{'max_depth': 1, 'max_features': 2, 'min_samples_leaf': 15, 'n_estimators': 10}
accuracy Score (testing_gridsearch) after hypertuning for Random Forest:0.961449
Confusion Matrix after hypertuning for Random Forest:
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.96      1.00      0.98     12420
         Yes       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918

[0.867      0.867      0.86686687]


